# Blocks World Problem Using A* Algorithm
#### Irfhana Zakir Hussain | RA1811027010100 | April 21 2021

## Aim
- To provide a programming implementation to the Blocks World Problem using A* Search algorithm and Manhatten Distance Heuristic

## Manual Procedure

### Initial State:
| A |               
| B | | D |             
| C | | E |

### Goal State

| E |    ---- | C |         
| A | | B | | D |

### Solution:

1. Move A on table

2. Move D on table

3. Move B on table

4. Move E on A

5. Move C on D

Each step is chosed using A* algorithm which will calculate the cost of moving a block (1) and the manhatten distance from the current state from the goal state.



## Code

In [116]:
from itertools import permutations
from time import perf_counter
from copy import deepcopy
import re

In [117]:
def blocksWorld(): #sets up the problem as described in the manual procedure

    blocks = ['E', 'C', 'D', 'A', 'B']
    initial_state = [('CLEAR', 'A', ''), ('CLEAR', 'D', ''), ('ONTABLE', 'C',''), ('ONTABLE', 'E', ''), 
    ('ON', 'A', 'B'), ('ON', 'B', 'C'), ('ON', 'D', 'E')]
    goal_state= [('E', 'A'), ('C','D')]
    setup = {i: ['table', True] for i in blocks}
    print('Initial: ', initial_state)
    print('Goal: ', goal_state)
    print('Blocks: ', blocks)
    for block in initial_state:
        if block[0] == 'ON':
            setup[block[1]][0] = block[2]
            setup[block[2]][1] = False
    goal = {i: ['table', True] for i in blocks}
    for block in goal_state:
        goal[block[0]][0] = block[1]
        goal[block[1]][1] = False
    return setup, goal, blocks

In [118]:
class Node(object):
    """
        definition 
            Each node in the tree is a state. The definition explains each                 block in the state in the following manner:
            'Block name': ['On Top of Block Name', Movable? (Boolean)]
        parent
            The parent node

        move
            Describes the move that needed to take place to get to this state 
            from the parent node
            [Moved Block Name, From Block Name/Table, To Block Name/Table]
    """

    def __init__(self, definition=None, parent=None, move=None):
        super(Node, self).__init__()
        self.parent = parent
        self.pastmove = move 
        if not definition:
            self.defi = deepcopy(self.parent.defi)
            # If that move doesn't exists, it's a root state.
            if self.pastmove is not None:
                self.move(self.pastmove[0], self.pastmove[2])
        else:
            self.defi = definition
    def __hash__(self):
        string = ''
        for key, value in self.defi.items():
            string += "".join(key + value[0] + str(value[1])[0])
        return hash(string)
    def __eq__(self, other):
        if other is None:
            return False
        return self.defi == other.defi
    def __repr__(self):
        return str(self.defi) + '\n'
    def getpossmoves(self):
        movable_blocks = [key for key in self.defi if self.defi[key][1] is True]
        # Calculate all possible move permutations and add the special case of moving a cube onto table, if it's not already.
        possibleMoves = list(permutations(movable_blocks, 2)) + [(block, 'table') for block in movable_blocks if self.defi[block][0] != 'table']
        poss_next_states = []
        for source, dest in possibleMoves:
            poss_next_states.append(Node(parent=self, move=self.move(source, dest, True)))
        return poss_next_states

    def move(self, object, dest, fake = False):
        old = self.defi[object][0] 
        if fake:
            return [object, old, dest]
        if old != 'table':
            self.defi[old][1] = True #the block that it used to be on is now movable!
        self.defi[object][0] = dest #the block is now on the dest block
        if dest != 'table':
            self.defi[dest][1] = False #the dest block is not movable now!
        move = [object, old, dest]
    
        return move

    def getPath(self):
        path = []
        current = self
        while current.parent is not None:
            path.append(current.pastmove)
            current = current.parent
        return path[::-1]

In [119]:
def heuristic(poss_next_states, goal):
    heuristic_scores = []
    for state in poss_next_states:
        wrong_pos = 0
        for block in state.defi:
            if state.defi[block] != goal.defi[block]:
                wrong_pos += 1
        heuristic_scores.append(wrong_pos + len(state.getPath()))
    return heuristic_scores.index(min(heuristic_scores))

def A_search(initial, goal):
    iterr = 0
    visit, list = set(), [initial]
    t1 = perf_counter()
    while list:
        t2 = perf_counter()
        if t2 - t1 > 60: #making it more computationally efficient in case no solution is possible
            return None, iterr
        iterr += 1
        item = heuristic(list, goal)
        best_node = list.pop(item)

        if best_node == goal:
            return best_node.getPath(), iterr

        for nextt in best_node.getpossmoves():
            if nextt in visit:
                continue
            visit.add(nextt)
            list.append(nextt)

In [120]:
def print_path(moves): #prints solution
    print('\n')
    i = 0
    for move in moves:
        i += 1
        print('{}. Move({}, {}, {})\n' .format(
            i, move[0], move[1], move[2]))

In [121]:
setup, goal, blocks = blocksWorld()
initial_state = Node(setup)
goal_state = Node(goal)
t1 = perf_counter()
path, iters = A_search(initial_state, goal_state)
t2 = perf_counter()
if path:
    print("\n\nCost: " + str(len(path)))
    print_path(path)
else:
    print("Solution Not Found!")

Initial:  [('CLEAR', 'A', ''), ('CLEAR', 'D', ''), ('ONTABLE', 'C', ''), ('ONTABLE', 'E', ''), ('ON', 'A', 'B'), ('ON', 'B', 'C'), ('ON', 'D', 'E')]
Goal:  [('E', 'A'), ('C', 'D')]
Blocks:  ['E', 'C', 'D', 'A', 'B']


Cost: 5


1. Move(D, E, table)

2. Move(A, B, table)

3. Move(E, table, A)

4. Move(B, C, table)

5. Move(C, table, D)



## Results

We have successfully implemented A* Search and Manhatten Distance Heuristic to solve the given blocks world problem.